# Lab Exercise 04

In [1]:
__author__ = "Víctor Vega Sobral"

---

# Local Part

---

## 1. Local Speech to Text 

In [2]:
import whisper

### 1.1 Loading the Whisper Turbo model

In [3]:
model = whisper.load_model("turbo")

c:\Users\victo\miniconda3\envs\f1_strat_manager\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loca

### 1.2 Padding to 30 seconds

In [4]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("55_radio_47.mp3")
audio = whisper.pad_or_trim(audio)


### 1.3 Log-Mel spectogram

In [5]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)
# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


### 1.4 Decoding the audio

In [6]:
# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)
# print the recognized text
print(result.text)

Rain expected in 4 minutes. What is the rain intensity? 2 to 3.


---

## 2. Local Text to Speech

Using ``Coqui TTS``, a fork of Mozilla TTS.

In [12]:
from TTS.api import TTS

### 2.1 Initializing TTS model

In [13]:
# Tacotron2
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=True, gpu=False)

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\TTS\utils\io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locatio

Removing weight norm...


### 2.2 Converting text to speech

In [14]:
tts.tts_to_file(text = "Rain expected in 4 minutes. What is the rain intensity? 2 to 3.", file_path="outputs/output.mp3")

 > Text splitted to sentences.
['Rain expected in 4 minutes.', 'What is the rain intensity?', '2 to 3.']
 > Processing time: 3.6516635417938232
 > Real-time factor: 0.5822571813646433


'outputs/output.mp3'

---

# External APIs services Part

### 1.1 Google Speech to Text External API

In [15]:
# pip install SpeechRecognition
import speech_recognition as sr

r = sr.Recognizer()
with sr.AudioFile("outputs/output.mp3") as source:
    audio = r.record(source)
    # This function calls the Google external API
    text = r.recognize_google(audio)  
    print(text)

rain expected in 4 minutes what is the rain intensity 2203


### 1.2 Google Text to Speech External API

In [16]:
# pip install gTTS
from gtts import gTTS

tts = gTTS("This is a test for Google text to speech external api", lang="en")
tts.save("outputs/google_tts_output.mp3")

---

# Selected Speech Task: speech to speech translation

A easy pipeline for doing this task would be the following:

1. First, use Whisper to transcribe the audio to text.

2. Then, traduce the text (Spanish to English).

3. Finally, using mozilla TTS to create the new audio.

However, this approach adds a lot of latency for calling three models anc can be slow, so I will try to use ``NVIVIDA NeMo``, with a robust toolkit for speech-to_speech models based on Pytorch.

## Update: changing to first approach

NVIDIA NeMO is giving lots of problems from compilable tools that requiere WSL or a Linux distro. Then, I will move to the first approach even if it is not the most efficient one.

In [1]:
import torch
import whisper
from transformers import MarianMTModel, MarianTokenizer
from TTS.api import TTS

c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration: paths and model names
INPUT_AUDIO = "spanish_audio.mp3"         # Path to your Spanish mp3 file
OUTPUT_AUDIO = "english_translation.mp3"    # Path to save the translated English mp3
WHISPER_MODEL = "turbo"           # Whisper model size: tiny, base, small, medium, large
MT_MODEL = "Helsinki-NLP/opus-mt-es-en"
TTS_MODEL = "tts_models/en/ljspeech/tacotron2-DDC"


In [3]:
# Check device
USE_GPU = torch.cuda.is_available()
DEVICE = "cuda" if USE_GPU else "cpu"
print(f"[INFO] Using device: {DEVICE}")

[INFO] Using device: cpu


In [4]:
# 1) Load Whisper ASR model
asr_model = whisper.load_model(WHISPER_MODEL, device=DEVICE)

c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location

In [5]:
# 2) Load MarianMT tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(MT_MODEL)
mt_model = MarianMTModel.from_pretrained(MT_MODEL).to(DEVICE)

c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\victo\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-es-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this ar

In [6]:
# 3) Initialize Coqui TTS
tts = TTS(model_name=TTS_MODEL, progress_bar=False, gpu=USE_GPU)


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024


c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\TTS\utils\io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_locatio

 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Generator Model: hifigan_generator
 > Discriminator Model: hifigan_discriminator
Removing weight norm...


In [7]:
# -- ASR: Transcribe Spanish speech to text --
print(f"[ASR] Transcribing {INPUT_AUDIO}...")
asr_result = asr_model.transcribe(INPUT_AUDIO, language="es")
spanish_text = asr_result["text"].strip()
print(f"[ASR] Detected Spanish Text: {spanish_text}")



[ASR] Transcribing spanish_audio.mp3...


c:\Users\victo\miniconda3\envs\streamlitLLM\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[ASR] Detected Spanish Text: Hola, hola, hola, sacaba de Irla Lúth.


In [8]:
# -- MT: Translate Spanish text to English --
print(f"[MT] Translating to English...")
batch = tokenizer([spanish_text], return_tensors="pt", padding=True).to(DEVICE)
generated = mt_model.generate(**batch)
english_text = tokenizer.decode(generated[0], skip_special_tokens=True)
print(f"[MT] Translated English Text: {english_text}")


[MT] Translating to English...
[MT] Translated English Text: Hello, hello, hello, I was getting out of Irla Lúth.


In [ ]:
# -- TTS: Synthesize English text to speech --
print(f"[TTS] Generating English speech to {OUTPUT_AUDIO}...")
tts.tts_to_file(text=english_text, file_path=OUTPUT_AUDIO)
print(f"[TTS] Saved English audio at {OUTPUT_AUDIO}")

print("[INFO] Translation pipeline complete.")

[TTS] Generating English speech to english_translation.mp3...
 > Text splitted to sentences.
['Hello, hello, hello, I was getting out of Irla Lúth.']
hello, hello, hello, i was getting out of irla lúth.
 [!] Character 'ú' not found in the vocabulary. Discarding it.
